In [1]:
# Append rolling_ta to sys path.
import sys
import os

# Assuming rolling_ta is in the same level as tests/
module_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add the path to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from rolling_ta.data import CSVLoader
from rolling_ta.volume import NumbaOBV
from rolling_ta.logging import logger
from ta.volume import OnBalanceVolumeIndicator
from time import time

In [3]:
xls_loader = CSVLoader()
data = xls_loader.read_resource().copy()

DEBUG - CSVLoader: Loading from resources/btc_ohlcv.csv


In [4]:
# Precompile njit functions
NumbaOBV(data)

DEBUG - bytecode dump:
>          0	NOP(arg=None, lineno=128)
           2	RESUME(arg=0, lineno=128)
           4	LOAD_FAST(arg=0, lineno=133)
           6	LOAD_ATTR(arg=0, lineno=133)
          26	STORE_FAST(arg=2, lineno=133)
          28	LOAD_GLOBAL(arg=3, lineno=134)
          38	LOAD_FAST(arg=2, lineno=134)
          40	LOAD_GLOBAL(arg=4, lineno=134)
          50	LOAD_ATTR(arg=6, lineno=134)
          70	KW_NAMES(arg=1, lineno=134)
          72	CALL(arg=2, lineno=134)
          80	STORE_FAST(arg=3, lineno=134)
          82	LOAD_CONST(arg=2, lineno=135)
          84	LOAD_FAST(arg=3, lineno=135)
          86	LOAD_CONST(arg=3, lineno=135)
          88	STORE_SUBSCR(arg=None, lineno=135)
          92	LOAD_GLOBAL(arg=9, lineno=137)
         102	LOAD_ATTR(arg=10, lineno=137)
         122	LOAD_CONST(arg=4, lineno=137)
         124	LOAD_FAST(arg=2, lineno=137)
         126	CALL(arg=2, lineno=137)
         134	GET_ITER(arg=None, lineno=137)
>        136	FOR_ITER(arg=49, lineno=137)
        

In [5]:
start = time()
for i in range(10000):
    obv = OnBalanceVolumeIndicator(data["close"], data["volume"])
    obv.on_balance_volume()
logger.info("TIMING: [{:,.20f}s]".format(time() - start))

INFO - TIMING: [2.57432961463928222656s]


In [6]:
start = time()
for i in range(10000):
    NumbaOBV(data)
logger.info("TIMING: [{:,.20f}s]".format(time() - start))

INFO - TIMING: [0.65536689758300781250s]


In [6]:
slice_a = data.copy().iloc[:100]
slice_b = data.copy().iloc[100:]

In [ ]:
%%timeit
for _ in range(10):
    rolling = NumbaOBV(slice_a)
    for _, series in slice_b.iterrows():
        rolling.update(series)